# Web API

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
import tidy3d as td
import tidy3d.web as web


[13:56:16] WARNING  This version of Tidy3D was pip installed from the 'tidy3d-beta' repository on   ]8;id=570131;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/__init__.py\__init__.py]8;;\:]8;id=610672;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/__init__.py#100\100]8;;\
                    PyPI. Future releases will be uploaded to the 'tidy3d' repository. From now on,                
                    please use 'pip install tidy3d' instead.                                                       

           INFO     Using client version: 1.9.1                                                     ]8;id=688463;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/__init__.py\__init__.py]8;;\:]8;id=575252;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/__init__.py#115\115]8;;\

## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


[13:56:17] INFO     Created task 'webAPI' with task_id '956f4c0c-0153-4e12-ba84-c11fe94d652f'.        ]8;id=440738;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=124993;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#131\131]8;;\

Output()

[13:56:23] INFO     status = queued                                                                   ]8;id=949751;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=638674;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#282\282]8;;\

Output()

[13:56:28] INFO     status = preprocess                                                               ]8;id=618760;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=879024;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#275\275]8;;\

...

[13:56:41] INFO     status = success                                                                  ]8;id=304335;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=878296;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#350\350]8;;\

           INFO     downloading file "output/monitor_data.hdf5" to "data/sim.hdf5"                    ]8;id=518849;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=428014;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#673\673]8;;\

Output()

[13:56:45] INFO     loading SimulationData from data/sim.hdf5                                         ]8;id=487817;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=679914;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#472\472]8;;\

           WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=89536;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=8368;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../_autosummary/tidy3d.web.webapi.run.html#tidy3d.web.webapi.run) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../_autosummary/tidy3d.web.webapi.start.html) and then exit the session and load the results at a later time using [web.load](../_autosummary/tidy3d.web.webapi.load.html#tidy3d.web.webapi.load).


## Job Container

The [web.Job](../_autosummary/tidy3d.web.container.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../_autosummary/tidy3d.Simulation.html).

While [Job](../_autosummary/tidy3d.web.container.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call [.run()] so we will do that now.

In [4]:
# set the logging level to warning to reduce downloading messages.
td.config.logging_level = "warning"

# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")


Output()

Output()

Output()

Output()

Output()

[13:57:07] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=133364;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=121874;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

Another convenient thing about [Job](../_autosummary/tidy3d.web.container.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [5]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

[13:57:08] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=7422;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=349684;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../_autosummary/tidy3d.web.container.Batch.html) object.
The batch object is like a [Job](../_autosummary/tidy3d.web.container.Job.html), but stores task metadata for a series of simulations.

In [6]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


Output()

Output()

Output()

[13:57:14] Started working on Batch.                                                               ]8;id=359503;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=173350;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#383\383]8;;\

Output()

[13:57:31] Batch complete.                                                                         ]8;id=914650;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=499131;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#417\417]8;;\

When the batch is completed, the output is not a [SimulationData](../_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../_autosummary/tidy3d.web.container.BatchData.html).  The data within this [BatchData](../_autosummary/tidy3d.web.container.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [7]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[13:57:33] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=627146;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=73957;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

Output()

[13:57:35] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=897903;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=466504;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

Output()

[13:57:38] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=897759;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=536787;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

{'sim_0': 13540217.155681273, 'sim_1': 13540217.155681273, 'sim_2': 13540217.155681273}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../_autosummary/tidy3d.web.queue.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [8]:
batch_results = web.run_async(simulations=sims, verbose=verbose)

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

In [9]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[13:58:02] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=423218;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=993380;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

Output()

[13:58:04] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=234783;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=214676;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

Output()

[13:58:05] WARNING  Simulation final field decay value of 0.33 is greater than the simulation shutoff ]8;id=888370;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=188995;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\
                    threshold of 1e-05. Consider simulation again with large run_time duration for                 
                    more accurate results.                                                                         

{'sim_0': 13540217.155681273, 'sim_1': 13540217.155681273, 'sim_2': 13540217.155681273}
